# Filtros de imagen

Son operaciones que modifican una imagen teniendo en cuenta, para cada pixel del resultado, solo un pequeño entorno de la imagen de entrada.

Los filtros **lineales** se pueden expresar como la *convolución* de la imagen con una *máscara*.

Los no lineales son cualquier operación algorítmica sobre el entorno.

## Bibliotecas y utilidades

In [ ]:
import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt
import scipy.signal      as signal

In [ ]:
def fig(w,h):
    plt.figure(figsize=(w,h))

def readrgb(file):
    return cv.cvtColor( cv.imread("../images/"+file), cv.COLOR_BGR2RGB) 

def rgb2gray(x):
    return cv.cvtColor(x,cv.COLOR_RGB2GRAY)

def gray2float(x):
    return x.astype(float) / 255

# para ver imágenes monocromas autoescalando el rango
def imshowg(x):
    plt.imshow(x, 'gray')

# para ver imágenes monocromas de float con rango fijo
def imshowf(x):
    plt.imshow(x, 'gray', vmin = 0, vmax=1)

# para ver imágenes con signo
def imshows(x,r=1):
    plt.imshow(x, 'bwr', vmin = -r, vmax=r)

# ojo: filter2D no hace flip de la máscara (realmente hace correlación)
# (da igual en máscaras simétricas)
def conv(k,x):
    return cv.filter2D(x,-1,k)

# esta versión es correcta
def cconv(k,x):
    return signal.convolve2d(x, k, boundary='symm', mode='same')

## Convolución

Usamos una imagen cualquiera para ver el resultado

In [ ]:
rgb = readrgb("cube3.png")
g = rgb2gray(rgb)
f = gray2float(g)

plt.imshow(rgb);

Esta función recibe una máscara de convolución y compara la imagen original con el resultado del filtro.

In [ ]:
def democonv(k,x):
    print(k)
    fig(12,4)
    plt.subplot(1,2,1); imshowf(x); plt.title('original')
    plt.subplot(1,2,2); imshowf(cconv(k,x)); plt.title('resultado')

A partir de aquí probamos el efecto de diferentes máscaras:

In [ ]:
ker = np.array([[ 0, 0, 0]
               ,[ 0, 1, 0]
               ,[ 0, 0, 0]])
democonv(ker,f)

In [ ]:
ker = np.array([[ 0, 0, 0]
               ,[ 0, 3, 0]
               ,[ 0, 0, 0]])
democonv(ker,f)

In [ ]:
ker = np.array([[ 0, 0, 0]
               ,[ 0, .3, 0]
               ,[ 0, 0, 0]])
democonv(ker,f)

In [ ]:
ker = np.zeros([11,11])
ker[0,0] = 1
ker[10,10] = 1
ker = ker/np.sum(ker)

democonv(ker,f)

In [ ]:
ker = np.array([[ 0, 0, 0]
               ,[ -1, 0, 1]
               ,[ 0, 0, 0]])
democonv(ker,f)

Para visualizar mejor arrays cuyos elementos son floats con signo usamos un mapa de color (azul es negativo, blanco cero, y rojo positivo).

In [ ]:
def democonvs(k,x,s=1):
    print(k)
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1); imshowf(x); plt.title('original')
    plt.subplot(1,2,2); imshows(cconv(k,x),s); plt.title('resultado')

Derivada en dirección horizontal:

In [ ]:
ker = np.array([[ 0, 0, 0]
               ,[ -1, 0, 1]
               ,[ 0, 0, 0]])
democonvs(ker,f,0.2)

Derivada en dirección vertical:

In [ ]:
democonvs(ker.T,f,0.2)

Podemos combinar los dos anteriores para conseguir una medida de "borde" en cualquier orientación:

In [ ]:
def bordes(x):
    kx = np.array([[ 0, 0, 0]
                  ,[-1, 0, 1]
                  ,[ 0, 0, 0]])
    ky = kx.T
    gx = cconv(kx,x)
    gy = cconv(ky,x)
    return abs(gx)+abs(gy)

In [ ]:
imshowf(3*bordes(f))

El operador Laplaciano es la suma de las segundas derivadas respecto a cada variable:

$$\nabla^2 I = \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial y^2}$$

Su efecto es amplificar las frecuencias altas:

In [ ]:
imshows(cv.Laplacian(f,-1),0.05)

La siguiente máscara produce una aproximación al Laplaciano:

In [ ]:
democonvs(([[ 0, -1,  0]
           ,[ -1, 4, -1]
           ,[ 0, -1,  0]]), f, 0.05)

Se deduce de la aproximación a la derivada por diferencias finitas:

$$\frac{\partial I}{\partial x} \simeq I(x+1,y) - I(x,y)$$

In [ ]:
Dx = np.array([[-1,1]])

signal.convolve2d(Dx,Dx)

La suma de los coeficientes en las dos direcciones produce la máscara anterior.

Para calcular la derivadas suele utilizarse el operador de Sobel, cuyos coeficientes combinan automáticamente la diferencia de pixels con un leve suavizado de la imagen.

## Filtros de suavizado

La siguiente máscara calcula la **media** de un entorno de radio 5.

In [ ]:
ker = np.ones([11,11])
ker = ker/np.sum(ker)

democonv(ker,f)

Se consigue exactamente el mismo efecto con un "box filter".

In [ ]:
same = -1 # para que la imagen de salida sea del mismo tipo que la de entrada

imshowg(cv.boxFilter(f,same,(11,11)))

Lo interesante es que está implementado internamente usando "imágenes integrales", por lo que el tiempo de cómputo es constante, independientemente del tamaño de la región que se promedia.

In [ ]:
imshowg(cv.boxFilter(f,same,(30,30)))

In [ ]:
imshowg(cv.boxFilter(f,same,(300,300)))

No obstante, promediar un entorno abrupto de cada pixel produce "artefactos". La forma correcta de eliminar detalles es usar el filtro **gaussiano**, donde los pixels cercanos tienen más peso en el promedio.

In [ ]:
auto = (0,0) # tamaño de la máscara automático, dependiendo de sigma
sigma = 3

imshowg(cv.GaussianBlur(f, auto, sigma))

In [ ]:
imshowg(cv.GaussianBlur(f, auto, 20))

Es interesante observar el efecto en la imagen considerada como una superficie de niveles de gris:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(121, projection='3d')

r,c = g.shape
x,y = np.meshgrid(np.arange(c), np.arange(r))

# la coordenada z del gráfico 3D es el nivel de gris de la imagen anterior.
z = 255-g

ax.plot_surface(x,y,z, cmap='coolwarm', linewidth=0);
ax.view_init(60, 20)

ax = fig.add_subplot(122, projection='3d')
z = cv.GaussianBlur(255-g, auto, 10)

ax.plot_surface(x,y,z, cmap='coolwarm', linewidth=0);
ax.view_init(60, 20)

## Espacio de escala

El filtro gaussiano tiene varias características importantes:

- separable
- no introduce detalles: espacio de escala
- cascading
- Fourier
- analogía física

<video src='https://raw.githubusercontent.com/albertoruiz/umucv/master/images/demos/diffusion.mp4' controls='play'>scale space 1 </video>

<video src='https://raw.githubusercontent.com/albertoruiz/umucv/master/images/demos/gaucir.mp4' controls='play'>scale space 2 </video>

![scale space](http://raw.githubusercontent.com/albertoruiz/umucv/master/images/demos/scalespace.png)

![pyramid](http://raw.githubusercontent.com/albertoruiz/umucv/master/images/demos/pyramid.png)

## Convolución 1D como operación matricial

La operación de convolución con una máscara es realmente una forma compacta de expresar una operación lineal tradicional. En el caso unidimensional la correspondencia es sencilla:

In [ ]:
x = np.arange(12)
f = x % 4
plt.plot(f);
h = np.array([-1,2,-1])
r = np.convolve(h,f)
plt.plot(r[1:]);

In [ ]:
M = np.zeros([len(x)-2,len(x)])
for k in range(len(x)-2):
    M[k,k:k+3] = h
M

In [ ]:
plt.plot(M @ f);

Eso significa que en teoría es posible deshacer el efecto de un filtro lineal resolviendo un sistema de ecuaciones.

## Filtros no lineales

El filtro de **mediana** es no lineal. Es útil para eliminar ruido de "sal y pimienta", suavizando la imagen sin destruir los bordes. (Requiere pixels de tipo byte.)

In [ ]:
imshowg(cv.medianBlur(g,17))

El filtro **bilateral** solo promedia pixels cercanos que además tienen un valor similar.

In [ ]:
imshowg(cv.bilateralFilter(g,0,10,10))

In [ ]:
imshowg(cv.bilateralFilter(rgb,0,10,10))

Filtros de máximo y mínimo

In [ ]:
from scipy.ndimage import minimum_filter, maximum_filter

In [ ]:
imshowg(minimum_filter(g,11))

In [ ]:
imshowg(maximum_filter(g,11))

## Operadores morfológicos

(demo interactiva)

[ejemplos opencv](https://docs.opencv.org/master/d9/d61/tutorial_py_morphological_ops.html#gsc.tab=0)

[ejemplos skcikit image](http://scikit-image.org/docs/dev/auto_examples/applications/plot_morphology.html)

## Visualización interactiva

(Para que funcionen los siguientes apartados deben ejecutarse en un notebook normal.)

In [ ]:
# !pip install vispy jupyter_rfb

### Surface

In [ ]:
import numpy as np
import cv2 as cv

from vispy import app, scene
from vispy.io import imread

import os

image_filename = "../images/coins.png"

# Load images
img = imread(image_filename)

data = img[:,:,1]/255

# Create a canvas
canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)

# Add a view
view = canvas.central_widget.add_view()

# Grid dimensions
rows, cols = data.shape
x = np.arange(cols)/cols*2
y = np.arange(rows)/cols*2
x, y = np.meshgrid(x, y)

# Create vertices
z = data  # Use data for heights
vertices = np.stack([x.flatten(), y.flatten(), z.flatten()], axis=1)


# Create faces
faces = []
for i in range(rows - 1):
    for j in range(cols - 1):
        v0 = i * cols + j
        v1 = v0 + 1
        v2 = (i + 1) * cols + j
        v3 = v2 + 1
        faces.append([v0, v2, v3])
        faces.append([v0, v3, v1])
faces = np.array(faces)


# Create mesh
mesh = scene.visuals.Mesh(vertices=vertices, faces=faces, color=(.5, .7, .5, 1))
view.add(mesh)

#wireframe_filter = WireframeFilter(width=0.5)
#mesh.attach(wireframe_filter)

# Set camera
view.camera = 'turntable'

def update_surface(frame):
    vertices[:,2] = z = (frame[:,:,1]/255).flatten()
    colors = np.vstack([z,z,z]).T
    mesh.set_data(vertices = vertices, faces=faces, vertex_colors=colors)

update_surface(img)

canvas

In [ ]:
update_surface(cv.GaussianBlur(img,(0,0),2))

In [ ]:
from ipywidgets import interact

@interact(sigma=(1.,20))
def fun(sigma=1):
    update_surface(cv.GaussianBlur(img,(0,0),sigma))

### Pyramid

In [ ]:
import numpy as np
import cv2 as cv

from vispy import app, scene
from vispy.io import imread

import os

image_filename = "../images/coins.png"
img = imread(image_filename)
data = img[:,:,1]/255

canvas = scene.SceneCanvas(keys='interactive', bgcolor='w')
view = canvas.central_widget.add_view()
view.camera = scene.TurntableCamera(up='z', fov=60)

# Add a 3D axis to keep us oriented
axis = scene.visuals.XYZAxis(parent=view.scene)

#images = []

sc = 1
for k in np.arange(0,5,0.5):
    sigma = 2**k
    blurred = cv.GaussianBlur(img, (0,0), sigma)
    image = image1 = scene.visuals.Image(blurred, parent=view.scene)
    image.transform = scene.MatrixTransform()
    sc = 640
    image.transform.scale([1/sc,1/sc,1/sc])
    image.transform.translate([0,0,0.3*k])
    #images.append(image)

canvas

In [ ]:
import numpy as np
import cv2 as cv

from vispy import app, scene
from vispy.io import imread

import os

image_filename = "../images/coins.png"
img = imread(image_filename)

canvas = scene.SceneCanvas(keys='interactive', bgcolor='w')
view = canvas.central_widget.add_view()
view.camera = scene.TurntableCamera(up='z', fov=60)

# Add a 3D axis to keep us oriented
axis = scene.visuals.XYZAxis(parent=view.scene)

images = []

sc = 1
for k in np.arange(0,5,0.5):
    image = image1 = scene.visuals.Image(img, parent=view.scene)
    image.transform = scene.MatrixTransform()
    sc = 640 * 2**k
    d = 0.5-0.5**(k+1)
    image.transform.scale([1/sc,1/sc,1/sc])
    image.transform.translate([d,d,0.2*k])
    images.append(image)

canvas